# Skill 2 : Data Analysis from CSV file

To really have a Smart Search Engine or Virtual assistant that can answer any question about your corporate documents, this "engine" must understand tabular data, aka, sources with tables, rows and columns with numbers. 
This is a different problem that simply looking for the top most similar results.  The concept of indexing, bringing top results, embedding, doing a cosine semantic search and summarize an answer, doesn't really apply to this problem.
We are dealing now with sources with Tables in which each row and column are related to each other, and in order to answer a question, all of the data is needed, not just top results.

In this notebook, the goal is to show how to deal with this kind of use cases. To continue with our Covid-19 theme, we will be using an open dataset called ["Covid Tracking Project"](https://covidtracking.com/data/download). The COVID Tracking Project dataset is a  CSV file that provides the latest numbers on tests, confirmed cases, hospitalizations, and patient outcomes from every US state and territory (they stopped tracking on March 7 2021).

Imagine that many documents on a data lake are tabular data, or that your use case is to ask questions in natural language to a LLM model and this model needs to get the context from a CSV file or even a SQL Database in order to answer the question. A GPT Smart Search Engine, must understand how to deal with this sources, understand the data and answer acoordingly.

In [1]:
import os
import pandas as pd
from langchain_openai import AzureChatOpenAI
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent, create_pandas_dataframe_agent

from common.prompts import CSV_PROMPT_PREFIX

from IPython.display import Markdown, HTML, display  

from dotenv import load_dotenv
load_dotenv("credentials.env")

def printmd(string):
    display(Markdown(string))

In [2]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

## Download the dataset and load it into Pandas Dataframe

In [3]:
os.makedirs("data",exist_ok=True)

In [4]:
!wget https://covidtracking.com/data/download/all-states-history.csv -P ./data/

--2024-03-15 09:41:47--  https://covidtracking.com/data/download/all-states-history.csv
Resolving covidtracking.com (covidtracking.com)... 104.21.64.114, 172.67.183.132, 2606:4700:3032::ac43:b784, ...
Connecting to covidtracking.com (covidtracking.com)|104.21.64.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘./data/all-states-history.csv’

all-states-history.     [ <=>                ]   2.61M  --.-KB/s    in 0.05s   

2024-03-15 09:41:48 (49.1 MB/s) - ‘./data/all-states-history.csv’ saved [2738601]



In [5]:
file_url = "./data/all-states-history.csv"
df = pd.read_csv(file_url).fillna(value = 0)
print("Rows and Columns:",df.shape)
df.head()

Rows and Columns: (20780, 41)


date state    death  deathConfirmed  deathIncrease  deathProbable  \
0  2021-03-07    AK    305.0             0.0              0            0.0   
1  2021-03-07    AL  10148.0          7963.0             -1         2185.0   
2  2021-03-07    AR   5319.0          4308.0             22         1011.0   
3  2021-03-07    AS      0.0             0.0              0            0.0   
4  2021-03-07    AZ  16328.0         14403.0              5         1925.0   

   hospitalized  hospitalizedCumulative  hospitalizedCurrently  \
0        1293.0                  1293.0                   33.0   
1       45976.0                 45976.0                  494.0   
2       14926.0                 14926.0                  335.0   
3           0.0                     0.0                    0.0   
4       57907.0                 57907.0                  963.0   

   hospitalizedIncrease  ...  totalTestResults  totalTestResultsIncrease  \
0                     0  ...         1731628.0                         0   
1                     0  ...         2323788.0                      2347   
2                    11  ...         2736442.0                      3380   
3                     0  ...            2140.0                         0   
4                    44  ...         7908105.0                     45110   

   totalTestsAntibody  totalTestsAntigen  totalTestsPeopleAntibody  \
0                 0.0                0.0                       0.0   
1                 0.0                0.0                  119757.0   
2                 0.0                0.0                       0.0   
3                 0.0                0.0                       0.0   
4            580569.0                0.0                  444089.0   

   totalTestsPeopleAntigen  totalTestsPeopleViral  \
0                      0.0                    0.0   
1                      0.0              2323788.0   
2                 481311.0                    0.0   
3                      0.0                    0.0   
4                      0.0              3842945.0   

   totalTestsPeopleViralIncrease  totalTestsViral  totalTestsViralIncrease  
0                              0        1731628.0                        0  
1                           2347              0.0                        0  
2                              0        2736442.0                     3380  
3                              0           2140.0                        0  
4                          14856        7908105.0                    45110  

[5 rows x 41 columns]

Bad pipe message: %s [b'~z9\xc7j\x11F\xa0xX\x0f\xc7\x83\xd7\xbb9\x0f\x1e Ar\xf0\x18*\xf2\x89\xd6\xdb^\n?a\xe0\x10(\x99\xd7~\x8e\x80U\xb1j\xb5\xcdH+\xfb\x16\x9bY\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00']
Bad pipe message: %s [b"h\x80\xdb\x00lL\xcbt\xb8e\xb68;e\xd9\x8bj\xa4\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=\x00<\x005\x00/\x00\x9a\x00\x99\xc0\x07\xc0\x11\x00\x96\x00\x05\x00\xff\x01\x00\x00j\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00"]
Bad pipe message: %s [b'']
Bad pipe message: %s [b'\xd7%y\xa0\xc1\xd2\x87p\xae\x9bK\x01D\xe9\xa7\xa7\x96\xb5\x00\x00>\xc0\x14\xc0\n\x009\x008\x007\x006\xc0\x0f\xc

In [6]:
df.columns

Index(['date', 'state', 'death', 'deathConfirmed', 'deathIncrease',
       'deathProbable', 'hospitalized', 'hospitalizedCumulative',
       'hospitalizedCurrently', 'hospitalizedIncrease', 'inIcuCumulative',
       'inIcuCurrently', 'negative', 'negativeIncrease',
       'negativeTestsAntibody', 'negativeTestsPeopleAntibody',
       'negativeTestsViral', 'onVentilatorCumulative', 'onVentilatorCurrently',
       'positive', 'positiveCasesViral', 'positiveIncrease', 'positiveScore',
       'positiveTestsAntibody', 'positiveTestsAntigen',
       'positiveTestsPeopleAntibody', 'positiveTestsPeopleAntigen',
       'positiveTestsViral', 'recovered', 'totalTestEncountersViral',
       'totalTestEncountersViralIncrease', 'totalTestResults',
       'totalTestResultsIncrease', 'totalTestsAntibody', 'totalTestsAntigen',
       'totalTestsPeopleAntibody', 'totalTestsPeopleAntigen',
       'totalTestsPeopleViral', 'totalTestsPeopleViralIncrease',
       'totalTestsViral', 'totalTestsViralIncrease'

## CSV/Pandas Agent

LLMs are great for building question-answering systems over various types of data sources. In this section we’ll go over how to build Q&A systems over data stored in a CSV file(s). Like we did in the last notebook, the key to working with tabular CSV files is to give an LLM access to tools/experts for querying and interacting with the data

In [7]:
# Let's delve into a challenging question that demands a multi-step solution. 
# The path to solving it might not be immediately clear.
# When examining the dataframe above, even a human might struggle to determine which columns are pertinent.

QUESTION = """
How may patients were hospitalized during July 2020 in Texas. 
And nationwide as the total of all states? 
Use the hospitalizedIncrease column
"""

In [8]:
# First we load our LLM
COMPLETION_TOKENS = 1000
llm = AzureChatOpenAI(deployment_name=os.environ["GPT35_DEPLOYMENT_NAME"], 
                      temperature=0.5, max_tokens=COMPLETION_TOKENS)

Now we need our agent and our expert/tool.  
LangChain has created an out-of-the-box agents that we can use to solve our Q&A to CSV tabular data file problem.

In [9]:
agent_executor = create_pandas_dataframe_agent(llm=llm,
                                               df=df,
                                               verbose=True,
                                               agent_type="openai-tools")

In [10]:
%%time
try:
    display(Markdown(agent_executor.invoke(QUESTION)['output']))
except Exception as e:
    print(e)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[(df['date'].str.startswith('2020-07')) & (df['state'] == 'TX')]['hospitalizedIncrease'].sum()"}`


0
Invoking: `python_repl_ast` with `{'query': "df[df['date'].str.startswith('2020-07')]['hospitalizedIncrease'].sum()"}`


63105In July 2020, there were 0 patients hospitalized in Texas. Nationwide, the total number of patients hospitalized in July 2020 across all states was 63,105.

> Finished chain.


In July 2020, there were 0 patients hospitalized in Texas. Nationwide, the total number of patients hospitalized in July 2020 across all states was 63,105.

CPU times: user 196 ms, sys: 19.7 ms, total: 216 ms
Wall time: 2.76 s


That was good pandas code, and pretty fast. Let's add some prompt engineering to improve answer.

### Improving the Prompt

We can give it now an improved sager prompt. In `prompts.py` you can find `CSV_PROMPT_PREFIX`

In [11]:
CSV_PROMPT_PREFIX

'\n- First set the pandas display options to show all the columns, get the column names, then answer the question.\n- **ALWAYS** before giving the Final Answer, try another method. Then reflect on the answers of the two methods you did and ask yourself if it answers correctly the original question. If you are not sure, try another method.\n- \n- If the methods tried do not give the same result, reflect and try again until you have two methods that have the same result. \n- If you still cannot arrive to a consistent result, say that you are not sure of the answer.\n- If you are sure of the correct answer, create a beautiful and thorough response using Markdown.\n- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE, ONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**. \n- **ALWAYS**, as part of your "Final Answer", explain how you got to the answer on a section that starts with: "\n\nExplanation:\n". In the explanation, mention the column names that you used to get to the final answe

In [12]:
agent_executor = create_pandas_dataframe_agent(llm=llm,
                                               df=df,
                                               prefix=CSV_PROMPT_PREFIX,
                                               verbose=True,
                                               agent_type="openai-tools")

In [14]:
%%time
try:
    display(Markdown(agent_executor.invoke(QUESTION)['output']))
except Exception as e:
    print(e)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'df.columns'}`


Index(['date', 'state', 'death', 'deathConfirmed', 'deathIncrease',
       'deathProbable', 'hospitalized', 'hospitalizedCumulative',
       'hospitalizedCurrently', 'hospitalizedIncrease', 'inIcuCumulative',
       'inIcuCurrently', 'negative', 'negativeIncrease',
       'negativeTestsAntibody', 'negativeTestsPeopleAntibody',
       'negativeTestsViral', 'onVentilatorCumulative', 'onVentilatorCurrently',
       'positive', 'positiveCasesViral', 'positiveIncrease', 'positiveScore',
       'positiveTestsAntibody', 'positiveTestsAntigen',
       'positiveTestsPeopleAntibody', 'positiveTestsPeopleAntigen',
       'positiveTestsViral', 'recovered', 'totalTestEncountersViral',
       'totalTestEncountersViralIncrease', 'totalTestResults',
       'totalTestResultsIncrease', 'totalTestsAntibody', 'totalTestsAntigen',
       'totalTestsPeopleAntibody', 'totalTestsPeopleAntigen',
       'totalT

The number of patients hospitalized during July 2020 in Texas is 0, and nationwide the total hospitalized patients for all states during July 2020 is 63,105.

Explanation:
- To find the number of hospitalized patients in Texas during July 2020, we filtered the dataset for rows where the date contains '2020-07' and the state is 'TX'. Then we summed the values in the 'hospitalizedIncrease' column for Texas, which resulted in 0 hospitalizations.
- To find the nationwide total of hospitalized patients for all states during July 2020, we filtered the dataset for rows where the date contains '2020-07' and summed the values in the 'hospitalizedIncrease' column, resulting in 63,105 hospitalizations.

CPU times: user 488 ms, sys: 4.79 ms, total: 492 ms
Wall time: 5.31 s


## Evaluation
Let's see if the answer is correct

In [15]:
#df['date'] = pd.to_datetime(df['date'])
july_2020 = df[(df['date'] >= '2020-07-01') & (df['date'] <= '2020-07-31')]
texas_hospitalized_july_2020 = july_2020[july_2020['state'] == 'TX']['hospitalizedIncrease'].sum()
nationwide_hospitalized_july_2020 = july_2020['hospitalizedIncrease'].sum()

In [16]:
print( "TX:",texas_hospitalized_july_2020,"Nationwide:",nationwide_hospitalized_july_2020)

TX: 0 Nationwide: 63105


It is Correct!

**Note**: Obviously, there were hospitalizations in Texas in July 2020 (Try asking ChatGPT), but this particular File, for some reason has 0 on the column "hospitalizedIncrease" for Texas in July 2020. This proves though that the model is NOT making up information or using prior knowledge, but instead using only the results of its calculation on this CSV file. That's what we need!

# Summary

So, we just solved our problem on how to ask questions in natural language to our Tabular data hosted on a CSV File.
With this approach you can see then that it is NOT necessary to make a dump of a database data into a CSV file and index that on a Search Engine, you don't even need to use the above approach and deal with a CSV data dump file. With the Agents framework, the best engineering decision is to interact directly with the data source API without the need to replicate the data in order to ask questions to it. Remember, LLMs can do SQL very well. 


**Note**: We don't recommend using a pandas agent to answer questions from tabular data. It is not fast and it makes too many parsing mistakes. We recommend using SQL (see next notebook).

# NEXT
We can see that GPT-3.5, and even better GPT-4, is powerful and can translate a natural language question into the right steps in python in order to query a CSV data loaded into a pandas dataframe. 
That's pretty amazing. However the question remains: **Do I need then to dump all the data from my original sources (Databases, ERP Systems, CRM Systems) in order to be searchable by a Smart Search Engine?**

The next Notebook answers this question by implementing a Question->SQL process and get the information from data in a SQL Database, eliminating the need to dump it out.